In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
import pandas as pd

spark = SparkSession.builder.appName("worldwide_box_office").getOrCreate()

df = pd.read_csv("worldwide_box_office_1977_2025.csv")
df = spark.createDataFrame(df)

# retirando as duplicatas nos nomes dos filmes
df = df.dropDuplicates(["movie_title"])

# separando a data que está em ANO-MES-DIA para ficar em colunas
df = df.withColumn("release_year", split(df["release_date"], "-")[0]) \
       .withColumn("release_month", split(df["release_date"], "-")[1]) \
       .withColumn("release_day", split(df["release_date"], "-")[2])

# separando os gêneros para melhor organização e análise (não representa que o gênero 1 é o principal)
df = df.withColumn("genre_01", split(df["genres"], " ")[0]) \
       .withColumn("genre_02", split(df["genres"], " ")[1]) \
       .withColumn("genre_03", split(df["genres"], " ")[2]) \
       .withColumn("genre_04", split(df["genres"], " ")[3]) \
       .withColumn("genre_05", split(df["genres"], " ")[4]) \
       .withColumn("genre_06", split(df["genres"], " ")[5]) \
       .withColumn("genre_07", split(df["genres"], " ")[6]) \
       .withColumn("genre_08", split(df["genres"], " ")[7]) \
       .withColumn("genre_09", split(df["genres"], " ")[8]) \
       .withColumn("genre_10", split(df["genres"], " ")[9]) 

#df.show(5)

#spark.stop()

In [ ]:
from pyspark.sql.functions import year, avg, format_number, col, round

# Lendo o arquivo CSV com os dados do CPI (Consumer Price Index)
df_cpi = spark.read.csv("CPIAUCNS.csv", header=True, inferSchema=True)

# Convertendo a coluna observation_date para o tipo de dado Date 
cpi_df = df_cpi.withColumn("year_cpi", year("observation_date")) \
    .groupBy("year_cpi") \
    .agg(avg("CPIAUCNS").alias("CPI_value"))

# Arredondando para duas casas decimais pq não queria que a multiplicação ficasse com muitas casas decimais
cpi_df = cpi_df.withColumn('CPI_Value', format_number('CPI_Value', 2))

# (antes era para ser dinâmico, por isso renomeio a coluna, mas depois decidi deixar o valor fixo
# de 2024, isso aqui é um left join basicamente)
df = df.join(cpi_df.withColumnRenamed("CPI_value", "CPI_original"), df.release_year == cpi_df.year_cpi, "left")

# Ajustando os valores de bilheteira para o ano de 2024 usando o CPI
CPI_2024 = 313.69
df = df.withColumn("domestic_gross_adjusted", col("domestic_gross") * (CPI_2024 / col("CPI_original")))
df = df.withColumn("international_gross_adjusted", col("international_gross") * (CPI_2024 / col("CPI_original")))
df = df.withColumn("total_gross_adjusted", col("total_gross") * (CPI_2024 / col("CPI_original")))
df = df.withColumn("domestic_opening_adjusted", col("domestic_opening") * (CPI_2024 / col("CPI_original")))

# Arredondando para long (inteiro), para evitar problemas com valores decimais
# talvez isso faça com que o meu passo anterior de arredondar seja inútil, mas fazer o que né kkk
df = df.withColumn('domestic_gross_adjusted', round('domestic_gross_adjusted').cast('long'))
df = df.withColumn('international_gross_adjusted', round('international_gross_adjusted').cast('long'))
df = df.withColumn('total_gross_adjusted', round('total_gross_adjusted').cast('long'))
df = df.withColumn('domestic_opening_adjusted', round('domestic_opening_adjusted').cast('long'))
